<a href="https://colab.research.google.com/github/rodrigogrigo/analise-requisitos/blob/main/bert_training_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y transformers tokenizers
!rm -rf /usr/local/lib/python*/dist-packages/transformers*
!rm -rf /usr/local/lib/python*/dist-packages/tokenizers*
!pip install transformers==4.37.2 datasets

Found existing installation: transformers 4.52.2
Uninstalling transformers-4.52.2:
  Successfully uninstalled transformers-4.52.2
Found existing installation: tokenizers 0.21.1
Uninstalling tokenizers-0.21.1:
  Successfully uninstalled tokenizers-0.21.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 55.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.


In [ ]:
!pip install peft==0.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
!pip install accelerate==0.26.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.7.0
    Uninstalling accelerate-1.7.0:
      Successfully uninstalled accelerate-1.7.0


In [ ]:
import transformers
print(transformers.__version__)

4.37.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from datasets import Dataset, concatenate_datasets
from transformers import (
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    AutoModelForMaskedLM,
    TrainingArguments,
    Trainer,
)
import torch

In [ ]:
# Caminhos
PASTA_DADOS = "/content/drive/MyDrive/projetoMestrado/datasets_all_processados/treinamento_model"
MODELO_BASE = "burakkececi/bert-software-engineering"
PASTA_SAIDA_MODELO = "/content/drive/MyDrive/projetoMestrado/models/custom/bert_treinado_custom"

os.makedirs(PASTA_SAIDA_MODELO, exist_ok=True)

# 1. Carregar os CSVs
def carregar_datasets_da_pasta(pasta):
    arquivos_csv = [f for f in os.listdir(pasta) if f.endswith('.csv')]
    datasets = []
    for arquivo in arquivos_csv:
        df = pd.read_csv(os.path.join(pasta, arquivo))
        if 'treated_description_bert' in df.columns:
            ds = Dataset.from_pandas(df[['treated_description_bert']].rename(columns={'treated_description_bert': 'text'}))
            datasets.append(ds)
    return concatenate_datasets(datasets)

print("Carregando dados...")
dataset_total = carregar_datasets_da_pasta(PASTA_DADOS)

# 2. Tokenização
print("Tokenizando textos...")
tokenizer = AutoTokenizer.from_pretrained(MODELO_BASE)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
    )

tokenized_dataset = dataset_total.map(tokenize_function, batched=True, remove_columns=["text"])

# 3. Dividir em treino e validação (90% treino, 10% validação)
print("Separando treino e validação...")
train_test_split_result = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split_result["train"]
val_dataset = train_test_split_result["test"]

# 4. Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15,
)

# 5. Modelo base
model = AutoModelForMaskedLM.from_pretrained(MODELO_BASE)

# 6. Configuração de treinamento
training_args = TrainingArguments(
    output_dir=PASTA_SAIDA_MODELO,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    #per_device_train_batch_size=16,
    per_device_train_batch_size=4,
    #num_train_epochs=3,
    num_train_epochs=1,
    #save_steps=10_000,
    save_total_limit=1,
    push_to_hub=False,
    logging_steps=500,
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 8. Treinamento
print("Iniciando treinamento...")
trainer.train()

# 9. Salvamento
print(f"Salvando modelo em {PASTA_SAIDA_MODELO}")
model.save_pretrained(PASTA_SAIDA_MODELO)
tokenizer.save_pretrained(PASTA_SAIDA_MODELO)

Carregando dados...


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Tokenizando textos...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rodrigogrigoleto (rodrigogrigoleto-ifes-instituto-federal-do-esp-rito-santo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,2.109500,1.905415


Salvando modelo em /content/drive/MyDrive/projetoMestrado/models/custom/bert_treinado_custom


('/content/drive/MyDrive/projetoMestrado/models/custom/bert_treinado_custom/tokenizer_config.json',
 '/content/drive/MyDrive/projetoMestrado/models/custom/bert_treinado_custom/special_tokens_map.json',
 '/content/drive/MyDrive/projetoMestrado/models/custom/bert_treinado_custom/vocab.txt',
 '/content/drive/MyDrive/projetoMestrado/models/custom/bert_treinado_custom/added_tokens.json',
 '/content/drive/MyDrive/projetoMestrado/models/custom/bert_treinado_custom/tokenizer.json')